In [3]:
import pandas, copy

from tqdm import tqdm

pandas.options.display.max_columns=999

In [5]:
cog=pandas.read_csv('../create-coguk-dataset/cog_dataset-20210624.csv')

/opt/local/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (11,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Exploring the dataset

In [6]:
cog[:3]

,cogid,sequence_name,cog_country,adm1,is_pillar_2,sample_date,epi_week,lineage,lineages_version,lineage_conflict,lineage_ambiguity_score,scorpio_call,scorpio_support,scorpio_conflict,t1001i,n501y,d614g,del_21765_6,y453f,del_1605_3,a222v,n439k,mutations,p681h,q27stop,e484k,p323l,study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,instrument_platform,ena_country,submitted_ftp,read_count,base_count,object_store_bucket
0,PHWC-PYFPIR,Wales/PHWC-PYFPIR/2021,UK,UK-WLS,N,2021-05-19,73,None,PANGO-v1.2.13,NaN,NaN,NaN,NaN,NaN,I,Y,G,del,Y,ref,A,N,synSNP:C913T|synSNP:C1288T|synSNP:C3037T|orf1a...,H,*,E,L,PRJEB37886,SAMEA9132940,ERS6847341,ERX5658438,ERR6019078,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR601/ERR6019078/P...,96557.0,13320734.0,NaN
1,HSLL-160F062,England/HSLL-160F062/2021,UK,UK-ENG,Y,2021-06-03,75,B.1.617.2,PLEARN-v1.2.13,NaN,NaN,Delta (B.1.617.2-like),1.0,0.0,T,N,G,ref,Y,ref,A,N,synSNP:C3037T|orf1ab:A1306S|orf1ab:P2046L|orf1...,R,Q,E,L,PRJEB37886,SAMEA9148402,ERS6873131,ERX5686029,ERR6047046,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR604/ERR6047046/H...,3430083.0,721453740.0,NaN
2,QEUH-160DE00,Scotland/QEUH-160DE00/2021,UK,UK-SCT,Y,2021-06-03,75,B.1.617.2,PLEARN-v1.2.13,NaN,NaN,Delta (B.1.617.2-like),1.0,0.0,T,N,G,X,Y,ref,A,N,orf1ab:S443F|synSNP:C3037T|orf1ab:A1306S|synSN...,R,Q,E,L,PRJEB37886,SAMEA9202513,ERS6927129,ERX5695457,ERR6056590,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR605/ERR6056590/Q...,1343069.0,282390969.0,NaN


In [7]:
cog=cog.loc[cog.object_store_bucket.notna()]

In [8]:
print("This gives us "+str(len(cog))+" COGUK samples with an ENA accession number and FASTQ files in the ena bucket in object store")

print("There are " +str(len(cog.loc[cog.lineage.isna()]))+" samples with no PANGO lineage and " +str(len(cog.loc[cog.scorpio_call.isna()]))+ " samples with no SCORPIO lineage")

This gives us 230879 COGUK samples with an ENA accession number and FASTQ files in the ena bucket in object store
There are 0 samples with no PANGO lineage and 155688 samples with no SCORPIO lineage


Let's look at the `pango` lineages and `scorpio` constellations

In [9]:
cog.lineage.value_counts(dropna=False)

B.1.1.7       75333
B.1.177       40367
B.1.1         14363
B.1            7304
B.1.177.7      5087
              ...  
B.1.1.136         1
B.1.177.14        1
C.1               1
B.1.1.411         1
B.1.319           1
Name: lineage, Length: 635, dtype: int64

In [10]:
cog.scorpio_call.value_counts(dropna=False)

NaN                           155688
Alpha (B.1.1.7-like)           74700
Beta (B.1.351-like)              208
A.23.1-like                       80
A.23.1-like+E484K                 57
Eta (B.1.525-like)                53
B.1.1.7-like+E484K                46
Zeta (P.2-like)                   30
B.1.1.318-like                     5
Epsilon (B.1.427/429-like)         5
Delta (B.1.617.2-like)             2
B.1.324.1-like                     2
B.1.617.1-like                     1
Gamma (P.1-like)                   1
Iota (B.1.526-like)                1
Name: scorpio_call, dtype: int64

What about the P.1 lineage?

In [11]:
cog.loc[cog.lineage.str[0]=='P'].scorpio_call.value_counts()

Zeta (P.2-like)     30
Gamma (P.1-like)     1
Name: scorpio_call, dtype: int64

In [12]:
cog.loc[cog.lineage.str[0]=='P'].lineage.value_counts()

P.2    32
P.1     1
Name: lineage, dtype: int64

In [13]:
cog.loc[cog.lineage=='None'].instrument_platform.value_counts()

ILLUMINA           15
OXFORD_NANOPORE     3
Name: instrument_platform, dtype: int64

And the newer Iota and Epsilon VUIs?

In [14]:
cog.loc[cog.scorpio_call=='Iota (B.1.526-like)'].lineage.value_counts()

B.1.526    1
Name: lineage, dtype: int64

In [15]:
cog.loc[cog.scorpio_call=='Epsilon (B.1.427/429-like)'].lineage.value_counts()

B.1.427/429    5
Name: lineage, dtype: int64

# Logic for creating testset

Let's arbitrarily choose the top 42 lineages, then add the others we want to sample and pick two samples from each.

I've picked 48 since that will fit onto a single `VM.Standard2.24` compute shape in OCI

This is all very arbitrary and can be changed!

In [16]:
a=cog.lineage.value_counts()
lineages_for_testset={}
lineages=list(a.loc[a>100][:50].index)
for i in lineages:
    lineages_for_testset[i]=4

lineages_for_testset['P.1']=1
lineages_for_testset['P.2']=4
lineages_for_testset    

lineages_for_testset['B.1.617.2']=2
lineages_for_testset['B.1.287']=1
lineages_for_testset['B.1.36.25']=1
lineages_for_testset['B.1.400']=1
lineages_for_testset['None']=4
# print(lineages_for_testset)
# len(lineages_for_testset)

samples=0
for i in lineages_for_testset:
    samples+=lineages_for_testset[i] 
print(samples)

214


In [22]:
len(lineages_for_testset)

57

## Algorithm

1. for a given lineage, randomly pick 10 samples (with replacement to deal with cases where there are fewer than 10 samples in that lineage)
2. for each, make a set out of its mutations ($A_i$)
3. for the samples stored to date, make a single set of all their collective mutations ($B$)
4. by considering $B-A_i$ in turn, choose the sample which has the most mutations not already in $B$
5. add to to the testset and repeat

In [17]:
def create_diverse_testset(coguk_df,required_lineages,random_seed=42,samples_to_pick=50):

    testset={}
    
    number_samples=0

    # iterate through the list of pango lineages we wish to cover in the testset
    for lineage in tqdm(lineages_for_testset):

        # before we pick some new samples to consider, build the set of existing mutations
        existing_mutations=set()    
        for idx in testset:    
            existing_mutations|=testset[idx]['mutations']

        # now 
        df=copy.deepcopy(coguk_df.loc[(coguk_df.lineage==lineage)])

        for n in range(lineages_for_testset[lineage]):
        
            try:
                random_samples=list(df.sample(n=samples_to_pick,random_state=random_seed,replace=True).index)
            except:
                print("No samples available for lineage "+lineage)
                continue 
            random_seed+=1

            sample_set={}
            additional_mutations=0
            next_sample=None

            for idx in random_samples:

                del1=cog.loc[idx].del_1605_3
                del2=cog.loc[idx].del_21765_6

                mutations=set()

                if del1=='X':
                    mutations|=set(['del_1605_3_X'])
                elif del1=='del':
                    mutations|=set(['del_1605_3'])

                if del2=='X':
                    mutations|=set(['del_21765_3_X'])
                elif del2=='del':
                    mutations|=set(['del_21765_3'])

                sample_set[idx]={}
                if isinstance(cog.loc[idx].mutations,str):
                    mutations|=set(cog.loc[idx].mutations.split('|'))      

                sample_set[idx]['mutations']=mutations
                sample_set[idx]['pango_lineage']=cog.loc[idx].lineage
                sample_set[idx]['scorpio_lineage']=cog.loc[idx].scorpio_call

            next_sample=random_samples[0]

            for idx in random_samples:    
                if idx in sample_set.keys() and len(sample_set[idx]['mutations']-existing_mutations)> additional_mutations:
                    next_sample=idx
                    additional_mutations=len(sample_set[idx]['mutations']-existing_mutations)
            try:
                testset[next_sample]=sample_set[next_sample]                
            except:
                print(next_sample,sample_set)

    muts=set()
    for i in testset:
        muts|=testset[i]['mutations']
        
    print("There are "+str(len(testset))+" samples in the testset with "+str(len(muts))+" unique mutations")
    
    return(testset.keys())




In [18]:
df=copy.deepcopy(cog.loc[(cog.instrument_platform=='OXFORD_NANOPORE')])

samples=create_diverse_testset(df,lineages_for_testset)

nanopore_testset=cog.loc[samples]

nanopore_testset.to_csv('cog-testset-genetically-diverse-'+str(len(nanopore_testset))+'samples-nanopore-v0.1.csv.gz')

nanopore_testset[['object_store_bucket','run_accession']].to_csv('sp3-testset-genetically-diverse-'+str(len(nanopore_testset))+'samples-nanopore-v0.1.csv',index=False,header=False)

 91%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊                              | 52/57 [00:12<00:00,  5.43it/s]

No samples available for lineage P.1
No samples available for lineage B.1.617.2
No samples available for lineage B.1.617.2
No samples available for lineage B.1.287
No samples available for lineage B.1.36.25
No samples available for lineage B.1.400


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 57/57 [00:12<00:00,  4.48it/s]

There are 158 samples in the testset with 1062 unique mutations


In [21]:
len(samples)

158

In [37]:
df=copy.deepcopy(cog.loc[(cog.instrument_platform=='ILLUMINA')])

samples=create_diverse_testset(df,lineages_for_testset)

illumina_testset=cog.loc[samples]

illumina_testset.to_csv('cog-testset-genetically-diverse-'+str(len(illumina_testset))+'samples-illumina-v0.1.csv.gz')

illumina_testset[['object_store_bucket','run_accession']].to_csv('sp3-testset-genetically-diverse-'+str(len(illumina_testset))+'samples-illumina-v0.1.csv',\
                                                    index=False,header=False)

100%|██████████| 57/57 [00:14<00:00,  3.90it/s]

There are 205 samples in the testset with 1431 unique mutations


In [23]:
illumina_testset=pandas.read_csv('cog-testset-genetically-diverse-205samples-illumina-v0.1.csv.gz')

In [24]:
foo=pandas.read_csv('cog-testset-genetically-diverse-48samples-illumina-v0.1.csv.gz')

In [30]:
duplicates=set(foo.cogid).intersection(set(illumina_testset.cogid))

In [35]:
illumina_testset=pandas.concat([foo[~foo.cogid.isin(duplicates)],illumina_testset])

In [36]:
illumina_testset

,Unnamed: 0,cogid,sequence_name,cog_country,adm1,is_pillar_2,sample_date,epi_week,lineage,lineages_version,lineage_conflict,lineage_ambiguity_score,scorpio_call,scorpio_support,scorpio_conflict,t1001i,n501y,d614g,del_21765_6,y453f,del_1605_3,a222v,n439k,mutations,p681h,q27stop,e484k,p323l,study_accession,sample_accession,secondary_sample_accession,experiment_accession,run_accession,instrument_platform,ena_country,submitted_ftp,read_count,base_count,object_store_bucket
0,256263,LOND-12F79EE,England/LOND-12F79EE/2021,UK,UK-ENG,Y,2021-01-21,56,B.1.526,PLEARN-v1.2.13,NaN,NaN,Iota (B.1.526-like),0.9474,0.0526,T,N,G,ref,Y,ref,A,N,orf1ab:T265I|synSNP:C3037T|orf1ab:G1946S|synSN...,P,Q,E,L,PRJEB37886,SAMEA7975947,ERS5663089,ERX5030303,ERR5226600,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR522/ERR5226600/L...,1246893.0,8.718694e+07,ena
1,303276,OXON-F41B12,England/OXON-F41B12/2020,UK,UK-ENG,Y,2020-12-13,51,B.1.1.311,PANGO-v1.2.13,NaN,NaN,NaN,NaN,NaN,T,N,G,ref,Y,ref,A,N,synSNP:C3037T|synSNP:C3775T|orf1ab:S2285F|synS...,P,Q,E,L,PRJEB37886,SAMEA7969548,ERS5656693,ERX5024073,ERR5220552,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR522/ERR5220552/O...,899334.0,2.085785e+08,ena
2,265937,MILK-1028272,England/MILK-1028272/2021,UK,UK-ENG,Y,2021-01-12,55,B.1.427/429,PANGO-v1.2.13,NaN,NaN,Epsilon (B.1.427/429-like),1.0000,0.0000,T,N,G,ref,Y,ref,A,N,orf1ab:T265I|synSNP:C2395T|synSNP:T2597C|synSN...,P,Q,E,L,PRJEB37886,SAMEA7870434,ERS5616890,ERX4995551,ERR5190704,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR519/ERR5190704/M...,3608408.0,7.590775e+08,ena
3,322114,MILK-C3084F,England/MILK-C3084F/2020,UK,UK-ENG,Y,2020-12-07,50,B.1.177.69,PANGO-v1.2.13,NaN,NaN,NaN,NaN,NaN,T,N,G,ref,Y,ref,V,N,synSNP:T445C|orf1ab:P371S|synSNP:C3037T|orf1ab...,P,Q,E,L,PRJEB37886,SAMEA7775117,ERS5522357,ERX4863894,ERR5057825,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR505/ERR5057825/M...,1633712.0,3.436849e+08,ena
4,248730,LOND-E4D56,England/LOND-E4D56/2021,UK,UK-ENG,N,2021-01-08,54,B.1.177.5,PANGO-v1.2.13,NaN,NaN,NaN,NaN,NaN,T,N,G,ref,Y,ref,V,N,synSNP:G370T|synSNP:T445C|synSNP:C3037T|synSNP...,P,Q,E,L,PRJEB37886,SAMEA8047933,ERS5735195,ERX5076634,ERR5286564,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR528/ERR5286564/L...,1135035.0,7.751911e+07,ena
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,107843,CAMC-142DAEA,England/CAMC-142DAEA/2021,UK,UK-ENG,Y,2021-03-18,64,B.1.617.2,PANGO-v1.2.13,NaN,NaN,Delta (B.1.617.2-like),1.0000,0.0000,T,N,G,ref,Y,ref,A,N,synSNP:T2278C|synSNP:C3037T|orf1ab:A1306S|orf1...,R,Q,E,L,PRJEB37886,SAMEA8508539,ERS6193345,ERX5371709,ERR5656218,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR565/ERR5656218/C...,5572858.0,1.171835e+09,demo_samples
201,436300,NORW-E92BC,England/NORW-E92BC/2020,UK,UK-ENG,N,2020-04-27,18,B.1.287,PLEARN-v1.2.13,NaN,NaN,NaN,NaN,NaN,T,N,G,ref,Y,ref,A,N,synSNP:C3037T|orf1ab:A2554V|orf1ab:P4715L|S:D6...,P,Q,E,L,PRJEB37886,SAMEA6964367,ERS4674226,ERX4204874,ERR4247022,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR424/ERR4247022/N...,837896.0,2.364005e+08,ena
202,336325,BHRT-24B59AB,England/BHRT-24B59AB/2020,UK,UK-ENG,Y,2020-11-01,45,B.1.36.25,PLEARN-v1.2.13,NaN,NaN,NaN,NaN,NaN,T,N,G,ref,Y,ref,V,N,synSNP:T445C|synSNP:C3037T|synSNP:C6286T|orf1a...,P,Q,E,L,PRJEB37886,SAMEA7782338,ERS5529569,ERX4871287,ERR5065179,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR506/ERR5065179/B...,3966003.0,5.380950e+08,ena
203,429185,OXON-B2094,England/OXON-B2094/2020,UK,UK-ENG,N,2020-04-13,16,B.1.400,PANGO-v1.2.13,NaN,NaN,NaN,NaN,NaN,T,N,G,ref,Y,ref,A,N,synSNP:C1714T|synSNP:C3037T|orf1ab:G993C|synSN...,P,Q,E,L,PRJEB37886,SAMEA7798063,ERS5545325,ERX4887026,ERR5080863,ILLUMINA,United Kingdom,ftp.sra.ebi.ac.uk/vol1/run/ERR508/ERR5080863/O...,55750.0,1.295440e+07,ena
